# LLM API — Getting Started

This notebook shows how to call the **GPT-OSS** model hosted on `hub.qazcode.ai`.

The server uses the **OpenAI-compatible API**, so you can use the `openai` Python library or plain `requests`.

## 1. Install dependencies

In [2]:
!uv add openai

Resolved 120 packages in 0.54ms
Audited 114 packages in 1ms


## 2. Configuration

In [ ]:
API_KEY = "YOUR_API_KEY"  # replace with your key
HUB_URL = "URL"

In [ ]:
from openai import OpenAI

client = OpenAI(
    base_url=HUB_URL,
    api_key=API_KEY,  # replace with your key
)

MODEL = "oss-120b"

## 3. Basic Chat Completion

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ],
)

print(response.choices[0].message.content)

## 4. System Prompt + User Message

In [ ]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a medical diagnosis assistant. Given patient symptoms, suggest the most probable diagnosis with an ICD-10 code."
        },
        {
            "role": "user",
            "content": "Patient presents with fever, dry cough, and shortness of breath lasting 5 days."
        }
    ],
)

print(response.choices[0].message.content)

## 5. Structured JSON Output

In [ ]:
import json

SYSTEM_PROMPT = """You are a clinical decision support system.
Given patient symptoms, return a JSON object:
{
  "diagnoses": [
    {"rank": 1, "icd_code": "...", "name": "...", "explanation": "..."},
    ...
  ]
}
Return top 3 diagnoses ranked by likelihood. Respond with JSON only."""

symptoms = "Severe headache, nausea, vomiting, neck stiffness, sensitivity to light."

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": symptoms},
    ],
)

result = json.loads(response.choices[0].message.content)
for d in result["diagnoses"]:
    print(f"[{d['rank']}] {d['icd_code']} — {d['name']}")
    print(f"    {d['explanation']}\n")

In [ ]:
import requests

response = requests.post(
    "https://{}/chat/completions".format(HUB_URL),
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "model": "oss-120b",
        "messages": [{"role": "user", "content": "Hello!"}],
    },
)

print(response.json()["choices"][0]["message"]["content"])